In [1]:
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

* **This requires plotly 4.0 or greater** *

### Dashapps to display:
1. implied volatility vs time and price using a constant underlying price

  * Calculate the implied volatility of each days options as if the underlying futures contract were  always some fixed value. For example, if the actual futures is 100, make that price 50, and then change all of the strike prices so that the strike prices are relative to the value 50.  

    1. The 100 call would be a 50 dollar call.  
    2. The 95 put would be a 45 dollar put.  
    3. The 105 call would be a 55 dollar call
   
2. Historical vs Implied Vol
3. General CSV Viewer and Grapher (graphing to come later)

In [2]:
import pandas as pd
import numpy as np
import datetime
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from dashapp import single_page_from_df as spfd
# from dashapp import dashapp2 as dashapp
import pyarrow as pa
import redis
import pdb
import pathlib


In [3]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [4]:
def get_redis_df(key):
    context = pa.default_serialization_context()
    df = context.deserialize(redis_db.get(key))
    return df

### Method to get constant volatility data from redis

In [5]:
def get_cuv_imp_from_redis():
    df_cuv_imp2 = get_redis_df('df_cuv_implied')
    df_cuv_imp2['commod'] = df_cuv_imp2.symbol.str.slice(0,2)
    df_cuv_imp2['contract_year'] = df_cuv_imp2.symbol.str.slice(-2,).astype(int) + 2000
    return df_cuv_imp2
        

In [ ]:
if __name__=='__main__':
    page_title = """Show ATM Implied Volatilities
    using a
    Constant Underlying Price"""
    # *************** Create other rows ****************************
    spfd.create_dashgraph_page(
        'df_cuv_implied',get_cuv_imp_from_redis,
        app_title="Constant Underlying Vol",
        page_title=page_title,
        app_port=8814,
        run=True,
    )


2020-08-12 12:07:41,200 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-08-12 12:07:41,228 - root - DEBUG - df_cuv_implied entering create_dt_div
2020-08-12 12:07:41,240 - root - DEBUG - df_cuv_implied exiting create_dt_div
2020-08-12 12:07:41,262 - root - INFO - This app will run at the URL: http://127.0.0.1:8814


Dash is running on http://127.0.0.1:8814/



2020-08-12 12:07:41,264 - dashapp.dashapp2 - INFO - Dash is running on http://127.0.0.1:8814/



2020-08-12 12:07:41,269 - dashapp.dashapp2 - INFO -  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.



...

2020-08-12 12:12:37,447 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:12:37] "POST /_dash-update-component HTTP/1.1" 200 -
2020-08-12 12:12:37,484 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:12:37] "POST /_dash-update-component HTTP/1.1" 200 -


needs quotes
_filter_query: commod == 'CL'
filter contains: 2019
_dash_table_update_paging_closure: page_current:24 page_size: 20


2020-08-12 12:12:48,559 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:12:48] "POST /_dash-update-component HTTP/1.1" 200 -
2020-08-12 12:12:48,577 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:12:48] "POST /_dash-update-component HTTP/1.1" 200 -


filter contains: CLF19
needs quotes
_filter_query: commod == 'CL'
filter contains: 2019
_dash_table_update_paging_closure: page_current:24 page_size: 20


2020-08-12 12:12:53,744 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:12:53] "POST /_dash-update-component HTTP/1.1" 200 -
2020-08-12 12:13:02,657 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:13:02] "POST /_dash-update-component HTTP/1.1" 200 -
2020-08-12 12:42:33,975 - werkzeug - INFO - 127.0.0.1 - - [12/Aug/2020 12:42:33] "POST /_dash-update-component HTTP/1.1" 200 -


###  Historical vs Implied Vol

In [ ]:
df_iv_final = pd.read_csv('./temp_folder/df_iv_final_CL.csv')

In [ ]:
df_iv_final2 = df_iv_final[['settle_date','symbol','atm_iv','close_y']]
df_iv_final2 = df_iv_final2.groupby(['settle_date','symbol'],as_index=False).last()
# df_iv_final3 = df_iv_final2[['settle_date','close_y']]
#df.groupby('id')['x'].rolling(2).mean()
# df_iv_final3['hist_vol'] = df_iv_final.close_y..apply.rolling(20).std()*256**.5
# df_iv_final3 = df_iv_final3[~df_iv_final3.hist_vol.isna()]


In [ ]:
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()


In [ ]:
def get_fut(symbol):
    sql = f"select * from {futtab} where symbol='{symbol}';"
    df_fut =  pga.get_sql(sql)
    df_fut = df_fut[['settle_date','symbol','close']]
    df_fut['hist_vol'] = df_fut.close.pct_change().rolling(20,min_periods=20).std()*256**.5
    df_fut = df_fut[~df_fut.hist_vol.isna()]
    return df_fut

In [ ]:
df_all_fut = None
for symbol in df_iv_final2.symbol.unique():
    df_fut = get_fut(symbol)
    if df_all_fut is None:
        df_all_fut = df_fut.copy()
    else:
        df_all_fut = df_all_fut.append(df_fut)

df_iv_final3 = df_iv_final2.merge(df_all_fut,how='inner',on=['settle_date','symbol'])
df_iv_final3.atm_iv = df_iv_final3.atm_iv.round(5)*100
df_iv_final3.hist_vol = df_iv_final3.hist_vol.round(5)*100
df_iv_final3['vol_diff'] = df_iv_final3.atm_iv - df_iv_final3.hist_vol

df_iv_final3        

In [ ]:
if __name__=='__main__':
    page_title = """Show ATM vs Historical"""
#     f = lambda:df_iv_final3
    f = spfd.dcc.Store(id='init_df_data_store',data=df_iv_final3.to_dict('records'))
    
    d = spfd.create_dashgraph_page(
        'df_iv_final3',f,
        app_title="Implied vs Historical",
        page_title=page_title,
        app_port=8814,
        run=True
    )


### App with CSV Uploader


In [ ]:
import importlib
importlib.reload(spfd)
importlib.reload(spfd.dashapp)
if __name__=='__main__':
    logger = spfd.dashapp.init_root_logger()    
    spfd.graph_from_csv_page(main_id='myid',app_port = 8814,app_title="csv_grapher",
        logger=logger)

2020-08-03 12:55:38,977 - root - DEBUG - myid_dtdf entering create_dt_div
2020-08-03 12:55:38,978 - root - DEBUG - myid_dtdf exiting create_dt_div
2020-08-03 12:55:38,989 - root - INFO - This app will run at the URL: http://127.0.0.1:8814


 * Serving Flask app "dashapp.dashapp2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-08-03 12:55:39,000 - werkzeug - INFO -  * Running on http://127.0.0.1:8814/ (Press CTRL+C to quit)
2020-08-03 12:56:23,195 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:23] "GET / HTTP/1.1" 200 -
2020-08-03 12:56:23,504 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:23] "GET /_dash-layout HTTP/1.1" 200 -
2020-08-03 12:56:23,509 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:23] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-08-03 12:56:23,533 - root - DEBUG - ****************************** execute_callback no data for [('myid_uploader_column_only_store_df', 'data')] ***************************************
2020-08-03 12:56:23,535 - root - DEBUG - ****************************** execute_callback no data for [('myid_uploader_file_path', 'children')] ***************************************
2020-08-03 12:56:23,536 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:23] "POST /_dash-update-component HTTP/1.1" 204 -
2020-08-03 12:56:23,537 - werkzeug - INFO - 127.0.0.1 

_dash_table_update_paging_closure: page_current:0 page_size: 100


2020-08-03 12:56:49,531 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:49] "GET / HTTP/1.1" 200 -
2020-08-03 12:56:49,990 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:49] "GET /_dash-layout HTTP/1.1" 200 -
2020-08-03 12:56:49,991 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:49] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-08-03 12:56:50,018 - root - DEBUG - ****************************** execute_callback no data for [('myid_uploader_column_only_store_df', 'data')] ***************************************
2020-08-03 12:56:50,022 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:50] "POST /_dash-update-component HTTP/1.1" 204 -
2020-08-03 12:56:50,021 - root - DEBUG - ****************************** execute_callback no data for [('myid_uploader_file_path', 'children')] ***************************************
2020-08-03 12:56:50,025 - werkzeug - INFO - 127.0.0.1 - - [03/Aug/2020 12:56:50] "POST /_dash-update-component HTTP/1.1" 204 -
2020-08-03 12:56:50,043 - root 

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

2020-08-03 12:58:35,524 - dashapp.dashapp2 - ERROR - Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py"

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view

2020-08-03 12:58:42,733 - dashapp.dashapp2 - ERROR - Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/flask/app.py"

In [ ]:
# ?spfd.html.Button

## END